In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision.models import ResNet, resnet18
from torchvision.models.utils import load_state_dict_from_url
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np
import os
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
train_preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset_train = torchvision.datasets.ImageFolder(root='./data/dog_cat/train', transform=train_preprocess)
loader_train = torch.utils.data.DataLoader(dataset_train,
                                           batch_size=64,
                                           shuffle=True,
#                                            num_workers=4
                                          ) 

dataset_test = torchvision.datasets.ImageFolder(root='./data/dog_cat/test', transform=test_preprocess)
loader_test = torch.utils.data.DataLoader(dataset_test,
                                          batch_size=64,
                                          shuffle=True,
#                                           num_workers=4
                                         ) 

In [3]:
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

In [4]:
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth'
}

def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)

class SEBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None, reduction=16):
        super(SEBasicBlock, self).__init__()
        super(SEBasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes, 1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.se = SELayer(planes, reduction)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.se(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        
        return out
    
def _resnet(arch, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

def se_resnet18(pretrained=False, progress=True, **kwargs):
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', SEBasicBlock, [2, 2, 2, 2], pretrained, progress,
                   **kwargs)

In [5]:
class Network(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.base_model = resnet18(pretrained=False)
        self.base_model.fc = nn.Linear(512, num_classes)

    def forward(self, x):       
        return self.base_model(x)

In [6]:
class SENetwork(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.base_model = se_resnet18(pretrained=False)
        self.base_model.fc = nn.Linear(512, num_classes)

    def forward(self, x):       
        return self.base_model(x)

In [7]:
clf_loss_func = torch.nn.CrossEntropyLoss()

In [8]:
gpu_flag = torch.cuda.is_available()
print(gpu_flag)
if gpu_flag:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

True


In [9]:
def train(model, loader, optimizer):
    model.train()
    correct = 0
    total = 0
    losses = []
    for X, y in tqdm(loader):
        X, y = X.to(device), y.to(device)
        clf = model(X)
        loss = clf_loss_func(clf, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        
        predict = clf.argmax(dim=1)
        correct += (predict == y.data).sum()
        total += len(y)
    
    return np.mean(losses), float(correct) / total

In [10]:
def valid(model, loader):
    model.eval()
  
    losses = []
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in tqdm(loader):
            X, y = X.to(device), y.to(device)
            clf = model(X)
            loss = clf_loss_func(clf, y)

            losses.append(loss.item())

            predict = clf.argmax(dim=1)
            correct += (predict == y).sum().item()
            total += len(y)
            
    return np.mean(losses), float(correct) / total

In [11]:
simple_model = Network(2)
se_model = SENetwork(2)
simple_model = simple_model.to(device)
se_model = se_model.to(device)

In [12]:
optimizer = torch.optim.SGD(simple_model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

best_loss = 1e+10
# best_state = None
earlystop_counter = 0
for epoch in range(20):
    train_loss, train_acc = train(simple_model, loader_train, optimizer)
    val_loss, val_acc = valid(simple_model, loader_test)

    if val_loss < best_loss:
        best_loss = val_loss
#         best_state = model.cpu().state_dict()

    scheduler.step()
    print('Epoch: {}'.format(epoch))
    print("train loss: {:.2f}, train acc: {:.2f}%".format(train_loss, train_acc*100.))
    print("val loss: {:.2f}, val acc: {:.2f}%".format(val_loss, val_acc*100.))

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 0
train loss: 0.77, train acc: 55.04%
val loss: 0.82, val acc: 52.50%


  0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 1
train loss: 0.69, train acc: 60.92%
val loss: 0.83, val acc: 57.00%


  0%|          | 0/79 [00:00<?, ?it/s]

...

Epoch: 17
train loss: 0.01, train acc: 99.90%
val loss: 0.81, val acc: 77.20%


  0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 18
train loss: 0.01, train acc: 100.00%
val loss: 0.81, val acc: 77.70%


100%|██████████| 16/16 [00:04<00:00,  3.28it/s]

Epoch: 19
train loss: 0.01, train acc: 99.92%
val loss: 0.83, val acc: 76.80%


In [13]:
optimizer = torch.optim.SGD(se_model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)
best_loss = 1e+10
# best_state = None
earlystop_counter = 0
for epoch in range(20):
    train_loss, train_acc = train(se_model, loader_train, optimizer)
    val_loss, val_acc = valid(se_model, loader_test)

    if val_loss < best_loss:
        best_loss = val_loss
#         best_state = model.cpu().state_dict()

    scheduler.step()
        
    print('Epoch: {}'.format(epoch))
    print("train loss: {:.2f}, train acc: {:.2f}%".format(train_loss, train_acc*100.))
    print("val loss: {:.2f}, val acc: {:.2f}%".format(val_loss, val_acc*100.))

  0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 0
train loss: 0.66, train acc: 60.42%
val loss: 0.96, val acc: 50.40%


  0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 1
train loss: 0.66, train acc: 63.12%
val loss: 0.63, val acc: 64.00%


  0%|          | 0/79 [00:00<?, ?it/s]

...

Epoch: 17
train loss: 0.01, train acc: 99.96%
val loss: 0.72, val acc: 79.50%


  0%|          | 0/79 [00:00<?, ?it/s]

Epoch: 18
train loss: 0.01, train acc: 99.94%
val loss: 0.75, val acc: 78.70%


100%|██████████| 16/16 [00:04<00:00,  3.21it/s]

Epoch: 19
train loss: 0.00, train acc: 100.00%
val loss: 0.74, val acc: 78.90%
